In [1]:
pip install nhlpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [187]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

from nhlpy.constants import BASE_URL
from nhlpy import team,game,schedule,player #There are other modules but this should do it for now

In [4]:
# Function that takes a game object created by nhlpy.game and 
# returns a 0 for a home team win and 1 for an away team win.
# Somehow, I could not find this data more directly using this
# package.

## Changed: add teams' names and abbrevs information 
## Changed: separated home and away dictionary keys

def whoWon(game):
    scoreData = game.all_stats()['liveData']['linescore']
    periodData = scoreData['periods']
    homegoals = 0
    awaygoals = 0
    
    # changed: 3 -> len(periodData) 
    # since not all games has 3 periods, eg.2019120001
    
    for i in range(len(periodData)):
        homegoals = homegoals + periodData[i]['home']['goals']
        awaygoals = awaygoals + periodData[i]['away']['goals']
    if homegoals == awaygoals:
        shootoutData = scoreData['shootoutInfo'] 
        homeshootout = shootoutData['home']['scores']
        awayshootout = shootoutData['away']['scores']
        winner = homeshootout < awayshootout
    else:
        winner = homegoals < awaygoals
    
    home_name = game.all_stats()['gameData']['teams']['home']['name']
    away_name = game.all_stats()['gameData']['teams']['away']['name']
    
    home_abbrev = game.all_stats()['gameData']['teams']['home']['abbreviation']
    away_abbrev = game.all_stats()['gameData']['teams']['away']['abbreviation']
    
    
    winner_dict = {'homeTeam': home_name,
                   'awayTeam': away_name,
                   'homeAbbrev': home_abbrev,
                   'awayAbbrev': away_abbrev,
                   'GamePK': game.all_stats()['gameData']['game']['pk'], 
                   'Winner': int(winner)}
    # return int(winner)
    return winner_dict

# A few notes: this will not work to determine ties (which happened prior to 2004),
# nor will it work in playoff games that go into overtime. I think it's okay for 
# now but I will need to fix if we include those games.

In [5]:
# Example game: Golden Knights @ Blue Jackets 1-4 (home team won)
exgame = game.Game(2017021023) # This number is the GamePk, the numerical code associated with a game
whoWon(exgame)

{'homeTeam': 'Columbus Blue Jackets',
 'awayTeam': 'Vegas Golden Knights',
 'homeAbbrev': 'CBJ',
 'awayAbbrev': 'VGK',
 'GamePK': 2017021023,
 'Winner': 0}

In [6]:
# This function takes a schedule object, and returns a dictionary
# with all of the information for the games in that season

# This was a method in nhlpy that is on Github, but not PyPI
# Defining it here as a function to make life easier
def season(schedule, seasonid=None):
    response = requests.get("{0}/schedule{1}{2}".format(BASE_URL, "?season=", seasonid))
    schedule.data = response.json()
    #del schedule.data["copyright"]
    return schedule.data

In [7]:
# Example season data:
thisschedule = schedule.Schedule()
thisseason = season(thisschedule,20202021)
print(thisseason.keys())
print(thisseason['totalGames'])
print(thisseason['dates'][0]['games'][4].keys())
print(thisseason['dates'][0]['games'][4]['gamePk'])


dict_keys(['copyright', 'totalItems', 'totalEvents', 'totalGames', 'totalMatches', 'metaData', 'wait', 'dates'])
952
dict_keys(['gamePk', 'link', 'gameType', 'season', 'gameDate', 'status', 'teams', 'venue', 'content'])
2020020005


In [8]:
# This is a function that gets all of the game pks from a given season
# Let's see if this is faster than hockey-scraper

def getPkList(season):
    numGames = season['totalGames']
    pklist = np.zeros(numGames)
    i = 0
    for date in season['dates']:
        for game in date['games']:
            pklist[i] = game['gamePk']
            i = i + 1
    return pklist

# In the future, it might make more sense to just grab everything we 
# need while looping through the season, but this should be a useful 
# tool for now.


In [9]:
# Example Pk List:
expklist = getPkList(thisseason)
expklist

array([2.02002000e+09, 2.02002000e+09, 2.02002000e+09, 2.02002000e+09,
       2.02002000e+09, 2.02002001e+09, 2.02002001e+09, 2.02002001e+09,
       2.02002001e+09, 2.02002001e+09, 2.02002001e+09, 2.02002001e+09,
       2.02002001e+09, 2.02002002e+09, 2.02002002e+09, 2.02002002e+09,
       2.02002002e+09, 2.02002002e+09, 2.02002002e+09, 2.02002002e+09,
       2.02002002e+09, 2.02002003e+09, 2.02002002e+09, 2.02002002e+09,
       2.02002003e+09, 2.02002003e+09, 2.02002003e+09, 2.02002003e+09,
       2.02002003e+09, 2.02002003e+09, 2.02002004e+09, 2.02002003e+09,
       2.02002004e+09, 2.02002004e+09, 2.02002004e+09, 2.02002004e+09,
       2.02002004e+09, 2.02002004e+09, 2.02002004e+09, 2.02002004e+09,
       2.02002004e+09, 2.02002004e+09, 2.02002005e+09, 2.02002005e+09,
       2.02002005e+09, 2.02002005e+09, 2.02002005e+09, 2.02002005e+09,
       2.02002005e+09, 2.02002006e+09, 2.02002006e+09, 2.02002006e+09,
       2.02002006e+09, 2.02002006e+09, 2.02002006e+09, 2.02002006e+09,
      

In [ ]:
# Win list for a season: perhaps this is what we regress onto?
# This doesn't work anymore
winlist = np.zeros(len(expklist))
for i, pk in enumerate(expklist):
    thisgame = game.Game(int(pk))
    win_dict = whoWon(thisgame)
    winlist[i] = win_dict['Winner']
    
# This loop takes me about one minute

In [ ]:
i


### create winner_hone_or_away csv dataset

In [ ]:
a = 20202021
for i in range(10):
    a = a-1
    a = a-10000
    print('creating season data: ', a)
    filename = str(a)+'_winner.json'

    thisschedule = schedule.Schedule()
    thisseason = season(thisschedule, a)
    expklist = getPkList(thisseason)

    season_stats = []
    for i, pk in enumerate(expklist):
        thisgame = game.Game(int(pk))
        season_stats.append(whoWon(thisgame))

    with open(filename, 'w') as outfile:
        json.dump(season_stats, outfile)
    outfile.close()



In [25]:
exgame.all_stats()['gameData']['game']['season']
exgame.boxscore()['teams']

{'away': {'team': {'id': 54,
   'name': 'Vegas Golden Knights',
   'link': '/api/v1/teams/54'},
  'teamStats': {'teamSkaterStats': {'goals': 1,
    'pim': 6,
    'shots': 38,
    'powerPlayPercentage': '0.0',
    'powerPlayGoals': 0.0,
    'powerPlayOpportunities': 2.0,
    'faceOffWinPercentage': '47.7',
    'blocked': 8,
    'takeaways': 6,
    'giveaways': 1,
    'hits': 21}},
  'players': {'ID8468674': {'person': {'id': 8468674,
     'fullName': 'Deryk Engelland',
     'link': '/api/v1/people/8468674',
     'firstName': 'Deryk',
     'lastName': 'Engelland',
     'primaryNumber': '5',
     'birthDate': '1982-04-03',
     'currentAge': 40,
     'birthCity': 'Edmonton',
     'birthStateProvince': 'AB',
     'birthCountry': 'CAN',
     'nationality': 'CAN',
     'height': '6\' 2"',
     'weight': 214,
     'active': True,
     'alternateCaptain': False,
     'captain': False,
     'rookie': False,
     'shootsCatches': 'R',
     'rosterStatus': 'Y',
     'currentTeam': {'id': 54,
    

In [11]:
from nhlpy import player

In [12]:
explayer = player.Player(8471214)
exdefenseman = player.Player(8476850)
exgoalie = player.Player(8475683)
print(explayer.season(2020,2021)['stats'][0]['splits'][0]['stat'])
print('goalie:')
print(exgoalie.season(2017,2018)['stats'][0]['splits'][0]['stat'])
print('defenseman')
print(exdefenseman.season(2017,2018)['stats'][0]['splits'][0]['stat'])

{'timeOnIce': '877:35', 'assists': 18, 'goals': 24, 'pim': 12, 'shots': 182, 'games': 45, 'hits': 98, 'powerPlayGoals': 9, 'powerPlayPoints': 17, 'powerPlayTimeOnIce': '189:41', 'evenTimeOnIce': '686:59', 'penaltyMinutes': '12', 'faceOffPct': 0.0, 'shotPct': 13.2, 'gameWinningGoals': 6, 'overTimeGoals': 1, 'shortHandedGoals': 0, 'shortHandedPoints': 0, 'shortHandedTimeOnIce': '00:55', 'blocked': 23, 'plusMinus': -7, 'points': 42, 'shifts': 935, 'timeOnIcePerGame': '19:30', 'evenTimeOnIcePerGame': '15:15', 'shortHandedTimeOnIcePerGame': '00:01', 'powerPlayTimeOnIcePerGame': '04:12'}
goalie:
{'timeOnIce': '3911:34', 'ot': 6, 'shutouts': 5, 'ties': 0, 'wins': 37, 'losses': 22, 'saves': 1835, 'powerPlaySaves': 207, 'shortHandedSaves': 41, 'evenSaves': 1587, 'shortHandedShots': 46, 'evenShots': 1698, 'powerPlayShots': 249, 'savePercentage': 0.920723, 'goalAgainstAverage': 2.423581, 'games': 65, 'gamesStarted': 65, 'shotsAgainst': 1993, 'goalsAgainst': 158, 'timeOnIcePerGame': '60:10', 'powe

In [29]:
homeroster = exgame.boxscore()['teams']['home']['players']
homeroster


{'ID8473914': {'person': {'id': 8473914,
   'fullName': 'Mark Letestu',
   'link': '/api/v1/people/8473914',
   'firstName': 'Mark',
   'lastName': 'Letestu',
   'primaryNumber': '55',
   'birthDate': '1985-02-04',
   'currentAge': 37,
   'birthCity': 'Elk Point',
   'birthStateProvince': 'AB',
   'birthCountry': 'CAN',
   'nationality': 'CAN',
   'height': '5\' 10"',
   'weight': 195,
   'active': True,
   'alternateCaptain': False,
   'captain': False,
   'rookie': False,
   'shootsCatches': 'R',
   'rosterStatus': 'Y',
   'currentTeam': {'id': 29,
    'name': 'Columbus Blue Jackets',
    'link': '/api/v1/teams/29'},
   'primaryPosition': {'code': 'C',
    'name': 'Center',
    'type': 'Forward',
    'abbreviation': 'C'}},
  'jerseyNumber': '55',
  'position': {'code': 'C',
   'name': 'Center',
   'type': 'Forward',
   'abbreviation': 'C'},
  'stats': {'skaterStats': {'timeOnIce': '13:06',
    'assists': 0,
    'goals': 0,
    'shots': 0,
    'hits': 0,
    'powerPlayGoals': 0,
    '

In [234]:
def gameRosterStats(gamepk):
    # All stats are normalized to per 60 minutes
    F_home_assists = []
    F_home_goals = []
    F_home_pim = []
    F_home_shots = []
    F_home_blocked = []
    F_home_hits = []
    F_home_pm = []
    F_home_atoi = []
    
    D_home_assists = []
    D_home_goals = []
    D_home_pim = []
    D_home_shots = []
    D_home_blocked = []
    D_home_hits = []
    D_home_pm = []
    D_home_atoi = []
    
    G_home_GAA = []
    G_home_atoi = []
    
    F_away_assists = []
    F_away_goals = []
    F_away_pim = []
    F_away_shots = []
    F_away_blocked = []
    F_away_hits = []
    F_away_pm = []
    F_away_atoi = []
    
    D_away_assists = []
    D_away_goals = []
    D_away_pim = []
    D_away_shots = []
    D_away_blocked = []
    D_away_hits = []
    D_away_pm = []
    D_away_atoi = []
    
    G_away_GAA = []
    G_away_atoi = []
    
    thisgame = game.Game(gamepk)
    try:
        hometeam = thisgame.all_stats()['gameData']['teams']['home']['triCode']
    except:
        hometeam = thisgame.all_stats()['gameData']['teams']['home']['abbreviation']
    homeroster = thisgame.boxscore()['teams']['home']['players']
    awayroster = thisgame.boxscore()['teams']['away']['players']
    season = thisgame.all_stats()['gameData']['game']['season']
    year = int(season[0:4])
    for thisplayer in homeroster.keys():
        explayer = player.Player(homeroster[thisplayer]['person']['id'])
        splits = explayer.season(year,year+1)['stats'][0]['splits']
        if splits:
            stats = splits[0]['stat']
        else:
            continue
        try:
            position = homeroster[thisplayer]['person']['primaryPosition']['code']
        except:
            position = homeroster[thisplayer]['position']['code']
        try:
            atoi_string = stats['timeOnIcePerGame']
        except:
            continue
        atoi_list = atoi_string.split(':')
        atoi_float = float(atoi_list[0])+float(atoi_list[1])/60
        toi_string = stats['timeOnIce']
        toi_list = toi_string.split(':')
        toi_float = float(toi_list[0])+float(toi_list[1])/60
        hours = toi_float/60
        if position in 'N/A':
            continue
        elif position in 'G':
            gaa = stats['goalAgainstAverage']
            G_home_GAA.append(gaa)
            G_home_atoi.append(atoi_float)
        else:
            assists = stats['assists']/hours
            goals = stats['goals']/hours
            shots = stats['shots']/hours
            blocked = stats['blocked']/hours
            hits = stats['hits']/hours
            pim = stats['pim']/hours
            pm = stats['plusMinus']/hours
            if 'D' in position:
                D_home_assists.append(assists)
                D_home_goals.append(goals)
                D_home_pim.append(pim)
                D_home_shots.append(shots)
                D_home_blocked.append(blocked)
                D_home_hits.append(hits)
                D_home_pm.append(pm)
                D_home_atoi.append(atoi_float)
            else:
                F_home_assists.append(assists)
                F_home_goals.append(goals)
                F_home_pim.append(pim)
                F_home_shots.append(shots)
                F_home_blocked.append(blocked)
                F_home_hits.append(hits)
                F_home_pm.append(pm)
                F_home_atoi.append(atoi_float)
                
    for thisplayer in awayroster.keys():
        explayer = player.Player(awayroster[thisplayer]['person']['id'])
        splits = explayer.season(year,year+1)['stats'][0]['splits']
        if splits:
            stats = splits[0]['stat']
        else:
            continue
        try:
            position = awayroster[thisplayer]['person']['primaryPosition']['code']
        except:
            position = awayroster[thisplayer]['position']['code']
        try:
            atoi_string = stats['timeOnIcePerGame']
        except:
            continue
        atoi_list = atoi_string.split(':')
        atoi_float = float(atoi_list[0])+float(atoi_list[1])/60
        toi_string = stats['timeOnIce']
        toi_list = toi_string.split(':')
        toi_float = float(toi_list[0])+float(toi_list[1])/60
        hours = toi_float/60
        if position in 'N/A':
            continue
        elif position in 'G':
            gaa = stats['goalAgainstAverage']
            G_away_GAA.append(gaa)
            G_away_atoi.append(atoi_float)
        else:
            assists = stats['assists']/hours
            goals = stats['goals']/hours
            shots = stats['shots']/hours
            blocked = stats['blocked']/hours
            hits = stats['hits']/hours
            pim = stats['pim']/hours
            pm = stats['plusMinus']/hours
            if 'D' in position:
                D_away_assists.append(assists)
                D_away_goals.append(goals)
                D_away_pim.append(pim)
                D_away_shots.append(shots)
                D_away_blocked.append(blocked)
                D_away_hits.append(hits)
                D_away_pm.append(pm)
                D_away_atoi.append(atoi_float)
            else:
                F_away_assists.append(assists)
                F_away_goals.append(goals)
                F_away_pim.append(pim)
                F_away_shots.append(shots)
                F_away_blocked.append(blocked)
                F_away_hits.append(hits)
                F_away_pm.append(pm)
                F_away_atoi.append(atoi_float)


    stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
                'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
                'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
                'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
                'F_home_blocked' : (np.array(F_home_blocked)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
                'F_home_hits' : (np.array(F_home_hits)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
                'F_home_pm' : (np.array(F_home_pm)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
                'D_home_assists' : (np.array(D_home_assists)*np.array(D_home_atoi)).sum()/sum(D_home_atoi),
                'D_home_goals' : (np.array(D_home_goals)*np.array(D_home_atoi)).sum()/sum(D_home_atoi),
                'D_home_pim' : (np.array(D_home_pim)*np.array(D_home_atoi)).sum()/sum(D_home_atoi),
                'D_home_shots' : (np.array(D_home_shots)*np.array(D_home_atoi)).sum()/sum(D_home_atoi),
                'D_home_blocked' : (np.array(D_home_blocked)*np.array(D_home_atoi)).sum()/sum(D_home_atoi),
                'D_home_hits' : (np.array(D_home_hits)*np.array(D_home_atoi)).sum()/sum(D_home_atoi),
                'D_home_pm' : (np.array(D_home_pm)*np.array(D_home_atoi)).sum()/sum(D_home_atoi),
                'G_home_GAA' : (np.array(G_home_GAA)*np.array(G_home_atoi)).sum()/sum(G_home_atoi),
                'F_away_assists' : (np.array(F_away_assists)*np.array(F_away_atoi)).sum()/sum(F_away_atoi),
                'F_away_goals' : (np.array(F_away_goals)*np.array(F_away_atoi)).sum()/sum(F_away_atoi),
                'F_away_pim' : (np.array(F_away_pim)*np.array(F_away_atoi)).sum()/sum(F_away_atoi),
                'F_away_shots' : (np.array(F_away_shots)*np.array(F_away_atoi)).sum()/sum(F_away_atoi),
                'F_away_blocked' : (np.array(F_away_blocked)*np.array(F_away_atoi)).sum()/sum(F_away_atoi),
                'F_away_hits' : (np.array(F_away_hits)*np.array(F_away_atoi)).sum()/sum(F_away_atoi),
                'F_away_pm' : (np.array(F_away_pm)*np.array(F_away_atoi)).sum()/sum(F_away_atoi),
                'D_away_assists' : (np.array(D_away_assists)*np.array(D_away_atoi)).sum()/sum(D_away_atoi),
                'D_away_goals' : (np.array(D_away_goals)*np.array(D_away_atoi)).sum()/sum(D_away_atoi),
                'D_away_pim' : (np.array(D_away_pim)*np.array(D_away_atoi)).sum()/sum(D_away_atoi),
                'D_away_shots' : (np.array(D_away_shots)*np.array(D_away_atoi)).sum()/sum(D_away_atoi),
                'D_away_blocked' : (np.array(D_away_blocked)*np.array(D_away_atoi)).sum()/sum(D_away_atoi),
                'D_away_hits' : (np.array(D_away_hits)*np.array(D_away_atoi)).sum()/sum(D_away_atoi),
                'D_away_pm' : (np.array(D_away_pm)*np.array(D_away_atoi)).sum()/sum(D_away_atoi),
                'G_away_GAA' : (np.array(G_away_GAA)*np.array(G_away_atoi)).sum()/sum(G_away_atoi),}
    return stat_dict

In [204]:
pk = expklist[5]

In [133]:
gameRosterStats(int(pk))

{'F_home_assists': 0.9704415730198707,
 'F_home_goals': 0.5941249059597126,
 'F_home_pim': 0.9010061409153846,
 'F_home_shots': 7.06184951834118,
 'F_home_blocked': 1.52076507313787,
 'F_home_hits': 2.6303368878766196,
 'F_home_pm': -1.0486036721651146,
 'D_home_assists': 0.5902156131479881,
 'D_home_goals': 0.23630766339352002,
 'D_home_pim': 1.8206688049012636,
 'D_home_shots': 4.459587843026419,
 'D_home_blocked': 3.785107476549193,
 'D_home_hits': 5.899630742697073,
 'D_home_pm': -0.8939214792241543,
 'G_home_GAA': 3.4681,
 'F_away_assists': 1.220369331985571,
 'F_away_goals': 1.0083251110903388,
 'F_away_pim': 2.1138351524951586,
 'F_away_shots': 7.057933987205377,
 'F_away_blocked': 1.637086437602514,
 'F_away_hits': 4.88409319520855,
 'F_away_pm': 0.1322961457262632,
 'D_away_assists': 0.8908402250353447,
 'D_away_goals': 0.22984229265211165,
 'D_away_pim': 1.2595442334956246,
 'D_away_shots': 4.17245667484033,
 'D_away_blocked': 3.759049149663783,
 'D_away_hits': 4.206850838447

In [100]:
exseason = '20202021'

In [235]:
thisschedule = schedule.Schedule()
thisseason = season(thisschedule,20212022)
pklist = getPkList(thisseason)
seasondata = pd.DataFrame(columns=['homeTeam','awayTeam','homeAbbrev','awayAbbrev','GamePK','Winner',
                                   'F_home_assists',
                                   'F_home_goals',
                                   'F_home_pim',
                                   'F_home_shots',
                                   'F_home_blocked',
                                   'F_home_pm',
                                   'D_home_assists',
                                   'D_home_goals',
                                   'D_home_pim',
                                   'D_home_shots',
                                   'D_home_blocked',
                                   'D_home_hits',
                                   'D_home_pm',
                                   'G_home_GAA',
                                   'F_away_assists',
                                   'F_away_goals',
                                   'F_away_pim',
                                   'F_away_shots',
                                   'F_away_blocked',
                                   'F_away_hits',
                                   'F_away_pm',
                                   'D_away_assists',
                                   'D_away_goals',
                                   'D_away_pim',
                                   'D_away_shots',
                                   'D_away_blocked',
                                   'D_away_hits',
                                   'D_away_pm',
                                   'G_away_GAA'])

lines = []
for i,pk in enumerate(pklist):
    this_dict = whoWon(game.Game(int(pk)))
    playerstats = gameRosterStats(int(pk))
    this_dict.update(playerstats)
    line = pd.DataFrame(this_dict, index = [0])
    seasondata = pd.concat([seasondata,line])
    print('game',i,'done')
    seasondata.to_csv('../Positions by Game/'+filename)
print(filename,"is done!")


game 0 done
game 1 done
game 2 done
game 3 done
game 4 done
game 5 done
game 6 done
game 7 done
game 8 done
game 9 done
game 10 done
game 11 done
game 12 done
game 13 done
game 14 done
game 15 done
game 16 done
game 17 done
game 18 done
game 19 done
game 20 done
game 21 done
game 22 done
game 23 done
game 24 done
game 25 done
game 26 done
game 27 done
game 28 done
game 29 done
game 30 done
game 31 done
game 32 done
game 33 done
game 34 done
game 35 done
game 36 done
game 37 done
game 38 done
game 39 done
game 40 done
game 41 done
game 42 done
game 43 done
game 44 done
game 45 done
game 46 done
game 47 done
game 48 done
game 49 done
game 50 done
game 51 done
game 52 done
game 53 done
game 54 done
game 55 done
game 56 done
game 57 done
game 58 done
game 59 done
game 60 done
game 61 done
game 62 done
game 63 done
game 64 done
game 65 done
game 66 done
game 67 done
game 68 done
game 69 done
game 70 done
game 71 done
game 72 done
game 73 done
game 74 done
game 75 done
game 76 done
game 77 d

game 594 done
game 595 done
game 596 done
game 597 done
game 598 done
game 599 done
game 600 done
game 601 done
game 602 done
game 603 done
game 604 done
game 605 done
game 606 done
game 607 done
game 608 done
game 609 done
game 610 done
game 611 done
game 612 done
game 613 done
game 614 done
game 615 done
game 616 done
game 617 done
game 618 done
game 619 done
game 620 done
game 621 done
game 622 done
game 623 done
game 624 done
game 625 done
game 626 done
game 627 done
game 628 done
game 629 done
game 630 done
game 631 done
game 632 done
game 633 done
game 634 done
game 635 done
game 636 done
game 637 done
game 638 done
game 639 done
game 640 done
game 641 done
game 642 done
game 643 done
game 644 done
game 645 done
game 646 done
game 647 done
game 648 done
game 649 done
game 650 done
game 651 done
game 652 done
game 653 done
game 654 done
game 655 done
game 656 done
game 657 done
game 658 done
game 659 done
game 660 done
game 661 done
game 662 done
game 663 done
game 664 done
game 6

game 1168 done
game 1169 done
game 1170 done
game 1171 done
game 1172 done
game 1173 done
game 1174 done
game 1175 done
game 1176 done
game 1177 done
game 1178 done
game 1179 done
game 1180 done
game 1181 done
game 1182 done
game 1183 done
game 1184 done
game 1185 done
game 1186 done
game 1187 done
game 1188 done
game 1189 done
game 1190 done
game 1191 done
game 1192 done
game 1193 done
game 1194 done
game 1195 done
game 1196 done
game 1197 done
game 1198 done
game 1199 done
game 1200 done
game 1201 done
game 1202 done
game 1203 done
game 1204 done
game 1205 done
game 1206 done
game 1207 done
game 1208 done
game 1209 done
game 1210 done
game 1211 done
game 1212 done
game 1213 done
game 1214 done
game 1215 done
game 1216 done
game 1217 done
game 1218 done
game 1219 done
game 1220 done
game 1221 done
game 1222 done
game 1223 done
game 1224 done
game 1225 done
game 1226 done
game 1227 done
game 1228 done
game 1229 done
game 1230 done
game 1231 done
game 1232 done
game 1233 done
game 1234 

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:163: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:164: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:165: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:166: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:167: Ru

game 1493 done


...

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:163: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:164: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:165: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:166: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:167: Ru

game 1502 done


C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:163: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:164: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:165: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:166: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:167: Ru

game 1503 done
game 1504 done
2005-2006_positions_by_game is done!


C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:163: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:164: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:165: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:166: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1004924947.py:167: Ru

In [231]:
thisgame = game.Game(int(pklist[0]))
homeroster = thisgame.boxscore()['teams']['home']['players']
players = [thisplayer for thisplayer in homeroster.keys()]
explayer = player.Player(homeroster[players[2]]['person']['id'])
explayer.season(2021,2022)

{'stats': [{'type': {'displayName': 'statsSingleSeason',
    'gameType': {'id': 'R',
     'description': 'Regular season',
     'postseason': False}},
   'splits': [{'season': '20212022',
     'stat': {'timeOnIce': '1480:03',
      'assists': 46,
      'goals': 34,
      'pim': 16,
      'shots': 256,
      'games': 81,
      'hits': 19,
      'powerPlayGoals': 13,
      'powerPlayPoints': 31,
      'powerPlayTimeOnIce': '244:33',
      'evenTimeOnIce': '1210:54',
      'penaltyMinutes': '16',
      'faceOffPct': 52.3,
      'shotPct': 13.3,
      'gameWinningGoals': 7,
      'overTimeGoals': 2,
      'shortHandedGoals': 0,
      'shortHandedPoints': 0,
      'shortHandedTimeOnIce': '24:36',
      'blocked': 30,
      'plusMinus': -9,
      'points': 80,
      'shifts': 1774,
      'timeOnIcePerGame': '18:16',
      'evenTimeOnIcePerGame': '14:56',
      'shortHandedTimeOnIcePerGame': '00:18',
      'powerPlayTimeOnIcePerGame': '03:01'}}]}]}

In [174]:
for col in seasondata.columns:
    print(col)
    print(np.sum(seasondata[col].values))

homeTeam
Philadelphia FlyersToronto Maple LeafsTampa Bay LightningEdmonton OilersColorado AvalancheBuffalo SabresNew Jersey DevilsNew York RangersDetroit Red WingsNashville PredatorsWinnipeg JetsEdmonton OilersArizona CoyotesVegas Golden KnightsLos Angeles KingsBuffalo SabresPhiladelphia FlyersTampa Bay LightningOttawa SenatorsColorado AvalancheNew Jersey DevilsArizona CoyotesNew York RangersDetroit Red WingsOttawa SenatorsEdmonton OilersNashville PredatorsLos Angeles KingsCalgary FlamesVegas Golden KnightsPittsburgh PenguinsFlorida PanthersDetroit Red WingsNew York IslandersToronto Maple LeafsPhiladelphia FlyersSt. Louis BluesNashville PredatorsCalgary FlamesEdmonton OilersAnaheim DucksVegas Golden KnightsNew York RangersPhiladelphia FlyersFlorida PanthersPittsburgh PenguinsOttawa SenatorsDetroit Red WingsLos Angeles KingsToronto Maple LeafsSt. Louis BluesAnaheim DucksVancouver CanucksVegas Golden KnightsBoston BruinsNew York IslandersColumbus Blue JacketsOttawa SenatorsVancouver Canu

In [207]:

years = [2010,2009,2008,2007,2006,2005]
for j in years:
    year = int(str(j)+str(j+1))
    filename = str(j)+'-'+str(j+1)+'_positions_by_game'
    thisseason = season(thisschedule,year)
    pklist = getPkList(thisseason)
    seasondata = pd.DataFrame(columns=['homeTeam','awayTeam','homeAbbrev','awayAbbrev','GamePK','Winner',
                                       'F_home_assists',
                                       'F_home_goals',
                                       'F_home_pim',
                                       'F_home_shots',
                                       'F_home_blocked',
                                       'F_home_pm',
                                       'D_home_assists',
                                       'D_home_goals',
                                       'D_home_pim',
                                       'D_home_shots',
                                       'D_home_blocked',
                                       'D_home_hits',
                                       'D_home_pm',
                                       'G_home_GAA',
                                       'F_away_assists',
                                       'F_away_goals',
                                       'F_away_pim',
                                       'F_away_shots',
                                       'F_away_blocked',
                                       'F_away_hits',
                                       'F_away_pm',
                                       'D_away_assists',
                                       'D_away_goals',
                                       'D_away_pim',
                                       'D_away_shots',
                                       'D_away_blocked',
                                       'D_away_hits',
                                       'D_away_pm',
                                       'G_away_GAA'])

    lines = []
    for i,pk in enumerate(pklist):
        teamdata = game.Game(int(pk)).all_stats()['gameData']['teams']
        if (teamdata['home']['id'] < 56) & (teamdata['away']['id'] < 56):  # Excluded exhibition games with teams outside the nhl
            this_dict = whoWon(game.Game(int(pk)))
            playerstats = gameRosterStats(int(pk))
            this_dict.update(playerstats)
            line = pd.DataFrame(this_dict, index = [0])
            seasondata = pd.concat([seasondata,line])
        else:
            continue
    seasondata.to_csv('../Positions by Game/'+filename)
    print(filename,"is done!")
        

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

2010-2011_positions_by_game is done!


...

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:157: RuntimeWarning: invalid value encountered in double_scalars
  stat_dict = {'F_home_assists' : (np.array(F_home_assists)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:158: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_goals' : (np.array(F_home_goals)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:159: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_pim' : (np.array(F_home_pim)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:160: RuntimeWarning: invalid value encountered in double_scalars
  'F_home_shots' : (np.array(F_home_shots)*np.array(F_home_atoi)).sum()/sum(F_home_atoi),
C:\Users\brian_n94aun6\AppData\Local\Temp\ipykernel_17060\1319334298.py:161: Ru

2005-2006_positions_by_game is done!


In [202]:
pk


2010010004.0

In [199]:
thisgame = game.Game(2010010004)
thisgame.boxscore()['teams']['home']['players']

{'ID8469638': {'person': {'id': 8469638,
   'fullName': 'Jussi Jokinen',
   'link': '/api/v1/people/8469638'},
  'jerseyNumber': '36',
  'position': {'code': 'N/A',
   'name': 'Unknown',
   'type': 'Unknown',
   'abbreviation': 'N/A'},
  'stats': {}},
 'ID8470988': {'person': {'id': 8470988,
   'fullName': 'Bryan Rodney',
   'link': '/api/v1/people/8470988'},
  'jerseyNumber': '28',
  'position': {'code': 'N/A',
   'name': 'Unknown',
   'type': 'Unknown',
   'abbreviation': 'N/A'},
  'stats': {}},
 'ID8473418': {'person': {'id': 8473418,
   'fullName': 'Stefan Chaput',
   'link': '/api/v1/people/8473418'},
  'jerseyNumber': '54',
  'position': {'code': 'N/A',
   'name': 'Unknown',
   'type': 'Unknown',
   'abbreviation': 'N/A'},
  'stats': {}},
 'ID8474822': {'person': {'id': 8474822,
   'fullName': 'Matthew Pistilli',
   'link': '/api/v1/people/8474822'},
  'jerseyNumber': '46',
  'position': {'code': 'N/A',
   'name': 'Unknown',
   'type': 'Unknown',
   'abbreviation': 'N/A'},
  'sta

In [137]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [140]:
logreg = LogisticRegression()
y = seasondata['Winner'].values
X = seasondata.iloc[:,6:]
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size = .2,
                                                   shuffle = True,
                                                   stratify = y,
                                                   random_state=973)

In [143]:
logreg.fit(X_train.values,y_train)
y_pred = logreg.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').